### Load the needed libraries

In [1]:
import requests
import yaml

### Load OpenAI API Key

In [2]:
with open('../secrets/credentials.yml', 'r') as stream:
    config = yaml.safe_load(stream)

OPENAI_CREDENTIALS = config['OPENAI_CREDENTIALS']

### Function to inference the OpenAI API

In [3]:
def query_openai(api_key, model, prompt_text, max_completion_tokens=100, temperature=0.7):
    """
    Queries an OpenAI chat model with customizable parameters and returns the response or a structured error message.

    :param api_key: Your OpenAI API key.
    :param model: The chat model to query (e.g., "gpt-3.5-turbo").
    :param prompt_text: The text prompt to send to the model.
    :param max_completion_tokens: The maximum number of tokens to generate in the completion.
    :param temperature: Controls randomness in the generation. Lower values make the model more deterministic.
    :return: The model's response or an error message.
    """
    url = "https://api.openai.com/v1/chat/completions"  
    headers = {"Authorization": f"Bearer {api_key}"}
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt_text}],
        "max_tokens": max_completion_tokens,
        "temperature": temperature,
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        if response.status_code == 200:
            data = response.json()
            # Extracting information from the response
            content = data['choices'][0]['message']['content']
            usage = data['usage']
            return (content, usage)
        
        else:
            error_message = f"HTTP Error {response.status_code}"
            try:
                error_details = response.json().get('error', {})
                message = error_details.get('message', 'An unspecified error occurred')
            except ValueError:
                message = "Error details unavailable"
            error_message += f": {message}"

            return {"error": error_message}
    except requests.RequestException as e:
        return {"error": f"Connection error: {e}"}


def calculate_inference_price(query_results, input_token_price, output_token_price):
    """
    Calculates the inference cost based on input and output token prices.

    Parameters:
    - query_results: A dictionary containing "prompt_tokens" and "completion_tokens" used in a query.
    - input_token_price: Price per input token.
    - output_token_price: Price per output token.

    Returns:
    - The total price for the inference based on the provided prices.
    """
    total_price = (query_results["prompt_tokens"] * input_token_price) + \
                  (query_results["completion_tokens"] * output_token_price)
    return total_price

### Example usage
##### Define the parameters

In [4]:
model = "gpt-3.5-turbo-0125"
input_token_price = 0.0000005
output_token_price = 0.0000015
max_completion_tokens = 1000  
temperature = 0.5 

##### Define the prompt 

In [5]:
prompt = """
Translate the following French text to Arabic: 
Saad LABRIJI
"""

##### Query OpenAI

In [6]:
response = query_openai(OPENAI_CREDENTIALS, model, prompt, max_completion_tokens, temperature)

print(f"Price in {calculate_inference_price(response[1], input_token_price, output_token_price)}$")
print("-------------")
print(response[0])

Price in 2.3500000000000002e-05$
-------------
سعد لبريجي


#### Function to prepare the Prompt 

In [7]:
def generate_contextual_rag_prompt(expertise_area, context):
    """
    Generates a detailed and engaging prompt tailored for a specific expertise area and context.

    :param expertise_area: The area of expertise to mention in the prompt (e.g., "textile design").
    :param context: The specific context or question to address (e.g., "I want to create a sustainable rag").
    :return: The crafted prompt string.
    """
    # Introduction with a call to action
    prompt = f"As an expert in {expertise_area}, your insights are invaluable. Your expertise is needed to address the following challenge:\n\n"

    # Setting the context
    prompt += f"Context: {context}\n\n"

    # Providing additional guidance and encouragement
    prompt += "Consider the following points when formulating your response:\n"
    prompt += "- Provide detailed suggestions and explanations.\n"
    prompt += "- Include examples and practical tips.\n"
    prompt += "- Explore innovative and eco-friendly approaches.\n\n"

    # Prompting for a comprehensive and insightful response
    prompt += "Your comprehensive answer will contribute significantly to solving this challenge.\n\n"

    # Encouragement to begin crafting the response
    prompt += "Start your response below:\n\n"

    return prompt

# Example usage
expertise_area = "textile design"
context = "What are the best materials and practices for creating an eco-friendly rag?"

result = generate_contextual_rag_prompt(expertise_area, context)

print(result)

As an expert in textile design, your insights are invaluable. Your expertise is needed to address the following challenge:

Context: What are the best materials and practices for creating an eco-friendly rag?

Consider the following points when formulating your response:
- Provide detailed suggestions and explanations.
- Include examples and practical tips.
- Explore innovative and eco-friendly approaches.

Your comprehensive answer will contribute significantly to solving this challenge.

Start your response below:


